In [0]:
import csv 
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as mpl
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.preprocessing import scale 
from google.colab import files 
from sklearn import preprocessing 
from random import seed
from random import randrange
from csv import reader

In [0]:
files.upload()

Saving RBF_dataset.csv to RBF_dataset (1).csv


{'RBF_dataset.csv': b'3,19,19,28,0\r\n3,5,11,2,0\r\n3,7,7,12,0\r\n3,7,12,20,0\r\n3,16,9,12,0\r\n3,17,13,11,0\r\n2,13,18,5,0\r\n3,6,12,11,0\r\n3,6,5,5,0\r\n1,3,6,19,0\r\n3,9,6,29,0\r\n4,20,19,10,1\r\n2,4,19,1,0\r\n5,4,7,22,1\r\n5,23,8,6,1\r\n4,8,8,18,0\r\n3,29,8,26,0\r\n2,25,18,2,0\r\n2,30,14,18,0\r\n2,27,11,27,0\r\n4,18,15,18,1\r\n4,16,7,20,0\r\n3,16,7,20,0\r\n3,3,9,22,0\r\n3,17,7,15,0\r\n4,26,19,3,1\r\n4,8,13,15,0\r\n3,11,17,12,0\r\n4,27,19,30,0\r\n4,5,14,21,0\r\n4,30,9,30,0\r\n4,16,13,25,0\r\n3,29,16,5,1\r\n3,27,12,14,0\r\n3,24,9,22,0\r\n3,25,5,11,0\r\n3,5,10,18,0\r\n4,25,19,2,1\r\n4,15,15,13,1\r\n3,11,11,4,0\r\n4,28,10,17,1\r\n4,20,5,28,0\r\n3,28,7,1,1\r\n3,27,11,21,0\r\n3,5,13,19,0\r\n4,14,9,30,0\r\n3,17,7,19,0\r\n4,17,8,7,1\r\n2,4,11,6,0\r\n3,13,7,20,0\r\n4,5,11,12,0\r\n3,6,15,3,0\r\n4,30,10,26,1\r\n3,12,17,7,0\r\n4,29,15,27,1\r\n3,15,20,1,1\r\n3,25,8,12,0\r\n2,28,10,9,0\r\n2,26,7,16,0\r\n3,13,9,14,0\r\n2,11,6,18,0\r\n4,10,14,29,0\r\n3,26,20,24,0\r\n4,9,12,3,1\r\n1,20,20,8,0\r\n3,

In [0]:
dataset=pd.read_csv('RBF_dataset (1).csv')
dataset

,3,19,19.1,28,0
0,3,5,11,2,0
1,3,7,7,12,0
2,3,7,12,20,0
3,3,16,9,12,0
4,3,17,13,11,0
...,...,...,...,...,...
4289,2,28,7,23,0
4290,3,23,5,9,0
4291,4,10,13,4,1
4292,5,16,8,9,1


In [0]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

In [0]:
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [0]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [0]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [0]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [0]:
# Make a prediction with weights
def predict(row, weights):
	activation = weights[0]
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0

In [0]:
# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
	return weights

In [0]:
# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)

In [0]:
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [0]:
seed(1)
# load and prepare data
filename = 'RBF_dataset (1).csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)
n_folds = 3
l_rate = 0.05
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [99.09154437456324, 97.48427672955975, 90.91544374563243]
Mean Accuracy: 95.830%
